In [1]:
from transformers import AutoTokenizer, AutoModel
import torch

In [2]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')
model = AutoModel.from_pretrained('sentence-transformers/bert-base-nli-mean-tokens')

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

/home/lzw365/anaconda3/envs/cse599/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
sentences = [
    "Three years later, the coffin was still full of Jello.",
    "The fish dreamed of escaping the fishbowl and into the toilet where he saw his friend go.",
    "The person box was packed with jelly many dozens of months later.",
    "He found a leprechaun in his walnut shell."
]

# initialize dictionary to store tokenized sentences
tokens = {'input_ids': [], 'attention_mask': []}

for sentence in sentences:
    # encode each sentence and append to dictionary
    new_tokens = tokenizer.encode_plus(sentence, max_length=128,
                                       truncation=True, padding='max_length',
                                       return_tensors='pt')
    tokens['input_ids'].append(new_tokens['input_ids'][0])
    tokens['attention_mask'].append(new_tokens['attention_mask'][0])

# reformat list of tensors into single tensor
tokens['input_ids'] = torch.stack(tokens['input_ids'])
tokens['attention_mask'] = torch.stack(tokens['attention_mask'])

In [6]:
outputs = model(**tokens)
type(outputs.keys())

odict_keys

In [7]:
embeddings = outputs.last_hidden_state

In [8]:
print(embeddings.shape)

torch.Size([4, 128, 768])


In [9]:
attention_mask = tokens['attention_mask']
print(attention_mask.shape)

torch.Size([4, 128])


In [12]:
mask = attention_mask.unsqueeze(-1).expand(embeddings.shape).float()
print(mask.shape)

torch.Size([4, 128, 768])


In [13]:
masked_embeddings = embeddings * mask

In [14]:
summed = torch.sum(masked_embeddings, 1)

In [15]:
summed_mask = torch.clamp(mask.sum(1), min=1e-9)

In [16]:
mean_pooled = summed / summed_mask

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
mean_pooled = mean_pooled.detach().numpy()

# calculate
cosine_similarity(
    [mean_pooled[0]],
    mean_pooled[1:]
)

array([[0.33088914, 0.7219259 , 0.55483633]], dtype=float32)